In [1]:
import os
import glob
import numpy as np
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, TimeDistributed, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import sparse_categorical_crossentropy
from sklearn.model_selection import train_test_split

In [2]:
def createDataset(path):
    video_list = []
    label = []
    for folder in os.listdir(f"{path}"):
        for video in glob.glob(os.path.join(f"{path}{folder}/*")):
            frame_list = []
            if folder=="0":
                vid = cv2.VideoCapture(video)
                while True:
                    success,frame = vid.read()
                    if not success:
                            break
                    frame = cv2.resize(frame,(50,50))
                    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                    frame = frame / 255.0
                    frame_list.append(frame)

                video_list.append(frame_list)
                label.append(0)

            else:
                vid = cv2.VideoCapture(video)
                while True:
                    success,frame = vid.read()
                    if not success:
                            break
                    frame = cv2.resize(frame,(50,50))
                    frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
                    frame = frame / 255.0
                    frame_list.append(frame)

                video_list.append(frame_list)
                label.append(1)
                
    label = np.array(label)
    return video_list, label[...,np.newaxis], max([len(i) for i in video_list])

In [3]:
X, Y, max_frame = createDataset("../input/joonedel/joon_del/")

In [4]:
for i,video in enumerate(X):
    if len(video)<max_frame:
        for j in range(max_frame-len(video)):
            frame = np.zeros((50,50,3))
            X[i].append(frame)
X = np.array(X)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2)

In [6]:
model = Sequential()
model.add(TimeDistributed(Conv2D(32,(3,3),activation="relu",input_shape=(X_train.shape[2:]))))
model.add(TimeDistributed(MaxPool2D()))
model.add(TimeDistributed(Conv2D(128,(3,3),activation="relu")))
model.add(TimeDistributed(MaxPool2D()))
model.add(TimeDistributed(Flatten()))

model.add(GRU(128,return_sequences=True))
model.add(GRU(64))
model.add(Dense(2,activation="softmax"))

2022-06-18 19:40:33.500340: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 19:40:33.586949: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 19:40:33.587679: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-18 19:40:33.588775: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [7]:
model.compile(optimizer=Adam(learning_rate=0.01),loss=sparse_categorical_crossentropy,metrics=["accuracy"])
checkpoint = ModelCheckpoint("CRNN.h5",monitor="val_accuracy",mode="max",save_best_only=True)

In [8]:
model.fit(X_train,Y_train,epochs=20,batch_size=4,validation_data=(X_test,Y_test),callbacks=[checkpoint])

2022-06-18 19:40:36.948211: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-06-18 19:40:40.594104: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


14/14 [==============================] - 13s 186ms/step - loss: 0.9338 - accuracy: 0.4074 - val_loss: 0.6931 - val_accuracy: 0.5714
Epoch 2/20
14/14 [==============================] - 2s 111ms/step - loss: 0.8580 - accuracy: 0.5000 - val_loss: 0.7072 - val_accuracy: 0.4286
Epoch 3/20
14/14 [==============================] - 2s 111ms/step - loss: 0.7215 - accuracy: 0.5000 - val_loss: 0.6931 - val_accuracy: 0.4286
Epoch 4/20
14/14 [==============================] - 2s 110ms/step - loss: 0.9617 - accuracy: 0.4444 - val_loss: 0.9631 - val_accuracy: 0.4286
Epoch 5/20
14/14 [==============================] - 2s 112ms/step - loss: 0.8780 - accuracy: 0.4815 - val_loss: 0.7188 - val_accuracy: 0.5714
Epoch 6/20
14/14 [==============================] - 2s 111ms/step - loss: 0.7520 - accuracy: 0.4815 - val_loss: 0.6808 - val_accuracy: 0.5714
Epoch 7/20
14/14 [==============================] - 2s 112ms/step - loss: 0.7513 - accuracy: 0.4630 - val_loss: 0.6814 - val_accuracy: 0.5714
Epoch 8/20
14/14

In [9]:
vid = cv2.VideoCapture("../input/joonedel/joon_del/0/013.mp4")
video_list=[]
while(True):
    success, frame = vid.read()
    if success == False:
        break
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame,(50,50))
    frame = frame / 255.
    video_list.append(frame)

In [10]:
for i in range(max_frame-len(video_list)):
    frame = np.zeros((50,50,3))
    video_list.append(frame)
video_list = np.array(video_list)
video = video_list[np.newaxis, ...]

In [11]:
pred = model.predict(video)
print(np.argmax(pred))

0
